In [1]:
import pandas as pd

from lets_plot import *

In [2]:
LetsPlot.setup_html()

In [3]:
wind_df = pd.read_csv("https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/pea_wind.csv")
records_count = wind_df.shape[0]

In [4]:
def is_float(x):
    try:
        float(x)
    except ValueError:
        return False
    return True

def update_wind_df():
    zero_drct_df = wind_df[wind_df["drct"] == 360.0]
    zero_drct_df.loc[:, "drct"] = 0
    return pd.concat([zero_drct_df, wind_df]).reset_index(drop=True)


wind_df = wind_df[wind_df['sped'].apply(lambda x: is_float(x))]
wind_df = wind_df[wind_df['drct'].apply(lambda x: is_float(x))]

wind_df['sped'] = wind_df['sped'].astype(float)
wind_df['drct'] = wind_df['drct'].astype(float)

wind_df = wind_df[wind_df['sped'].apply(lambda x: x >= 2.0)]

bins = [2, 5, 7, 10, 15, 20, float('inf')]
bin_ids = list(range(6))
wind_df['speed_group'] = pd.cut(wind_df['sped'], bins=bins, labels=bin_ids, right=False)

wind_df = update_wind_df()

wind_df.head()

,station,valid,drct,sped,speed_group
0,PEA,2002-05-17 10:15,0.0,8.05,2
1,PEA,2002-05-17 10:20,0.0,8.05,2
2,PEA,2002-05-17 10:35,0.0,5.75,1
3,PEA,2002-05-17 10:40,0.0,5.75,1
4,PEA,2002-05-17 19:25,0.0,5.75,1


In [16]:
ggplot(wind_df.iloc[99_900:100_000]) + \
    geom_density(
        aes(x='drct', fill=as_discrete('speed_group', order=1)), 
        position='stack', size=0,
        tooltips=layer_tooltips().format('^y', '{.2g}%').format('^x', '{}°')
    ) + \
    scale_fill_manual(
        name='Wind Speed (mph):', 
        values=['#002bff', '#03d3f8', '#7afe81', '#fde609', '#ff4404', '#780200'], 
        labels={
            0: '2 - 4.9', 
            1: '5 - 6.9', 
            2: '7 - 9.9', 
            3: '10 - 14.9', 
            4: '15 - 19.9', 
            5: '20+'
        },
    ) + \
    scale_x_continuous(
        labels={
            45:  'NE', 
            90:  'E', 
            135: 'SE', 
            180: 'S', 
            225: 'SW',
            270: 'W',
            315: 'NW',
            360: 'N', 
        },
    ) + \
    labs(
        title="Wind rose for [PEA] PELLA",
        subtitle="Observations from 2002 to 2024",
        caption="Data is provided by the Iowa Environmental Mesonet of Iowa State University"
    ) + \
    ggsize(800, 800) + \
    theme_minimal2() + \
    theme(
        plot_title=element_text(size=24, face='bold'),
        plot_subtitle=element_text(size=18),
        panel_grid_minor_x=element_line(),
        panel_grid=element_line(color='#A0A0A0'),
        axis_ticks_y=element_line(),
        axis_text_x=element_text(size=18),
        axis_title=element_blank()
    ) + \
    coord_polar(
        ylim=[-.002, None],
    )

In [24]:
ggplot({'x': [1, 1, 1, 2], 'g': ['a', 'a', 'b', 'b']}) + \
    geom_density(aes(x='x', fill='g'), position='stack') + \
    coord_polar()